##Cargando datos en el environment

Descargando datasets

In [3]:
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-1m.zip
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2021-05-14 14:30:58--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2021-05-14 14:30:58 (8.20 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

--2021-05-14 14:30:58--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip  

Descomprimiendo datasets

In [4]:
!unzip ml-latest-small.zip
!unzip ml-1m.zip
!unzip ml-25m.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


#Preparando dataset

In [79]:
import pandas as pd
import numpy as np
import numpy.ma as ma

# Movielens 100k
#df = pd.read_csv('ml-latest-small/ratings.csv')

# Movielens 1M
df = pd.read_csv('ml-1m/ratings.dat', sep=':', header=None)
df = df.drop(labels=[1,3,5], axis=1)
df.columns=['userId','movieId','rating','timestamp']

# Movielens 25M
#df = pd.read_csv('ml-25m/ratings.csv')

df.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Metadata de la tabla de rating

In [80]:
df.info

<bound method DataFrame.info of          userId  movieId  rating  timestamp
0             1     1193       5  978300760
1             1      661       3  978302109
2             1      914       3  978301968
3             1     3408       4  978300275
4             1     2355       5  978824291
...         ...      ...     ...        ...
1000204    6040     1091       1  956716541
1000205    6040     1094       5  956704887
1000206    6040      562       5  956704746
1000207    6040     1096       4  956715648
1000208    6040     1097       4  956715569

[1000209 rows x 4 columns]>

In [81]:
# Movielens 100k
#movieDf = pd.read_csv('ml-latest-small/movies.csv')

# Movielens 1M
movieDf = pd.read_csv('ml-1m/movies.dat', sep='::', engine='python', header=None)
movieDf.columns=['movieId','title','genres']

# Movielens 25M
#movieDf = pd.read_csv('ml-25m/movies.csv')
movieDf

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


#Preparando variables que se usarán a lo largo del programa

In [82]:
# movieIdName: Diccionario que tendrá como clave el id de una película
# y como valor el título de la película
movieIdName = {}
for index, row in movieDf.iterrows():
  movieIdName[row['movieId']] = row['title']

In [84]:
# calculando el tiempo manualmente para el dataset de 1M
#import time
#t0 = time.time()

# userRatings: Diccionario principal que contendrá como clave el id de un usuario
# y como valor tendrá otro diccionario, el cual tendrá como clave el id de una película
# y como valor tendrá el rating que dicho usuario le dió
userRatings = {}
for index, row in df.iterrows():
  movId = row['movieId'].astype(int)
  usrId  = row['userId'].astype(int)
  rating = row['rating']
  # Inicializando el diccionario para los rating de cada película de cada usuario
  if usrId in userRatings:
    userRatings[usrId][movId] = rating
  else:
    userRatings[usrId] = {movId: rating}
t1 = time.time()

#total = t1-t0

# Número de usuarios
print(len(userRatings))
#print(total)

6040
105.40069389343262


In [102]:
avg = 0
for usrId, ratings in userRatings.items():
  avg += len(ratings)
avg /= len(userRatings)
print(avg)

165.58632676709155


# Calculando las películas para recomendar a un usuario

In [85]:
# common_rating: Recibe 2 ids de usuario y filtra los rating que dieron
# para que solo se considere los casos en los que ambos le dieron un
# rating a una película
def common_rating(usrId1, usrId2):
  x = []
  y = []
  # Recorremos los rating del primer usuario
  for movId, rating in userRatings[usrId1].items():
    # Si el id de la película del rating del primer usuario también
    # se encuentra en los rating del segundo usuario, entonces
    # se agrega el rating del usuario 1 al array 'x' y
    # el rating del usuario 2 al array 'y'
    if movId in userRatings[usrId2]:
      x.append(rating)
      y.append(userRatings[usrId2][movId])
  # Se retornan como arreglos numpy para que el cálculo de medidas
  # de similitud sea más rápido
  return np.array(x), np.array(y)

In [86]:
# euclidean_distance: recibe 2 arreglos numpy
# y retorna su distancia euclidiana
def euclidean_distance(x, y):
  return np.sqrt(np.sum((x-y)**2))

# manhattan_distance: recibe 2 arreglos numpy
# y retorna su distancia manhattan
def manhattan_distance(x, y):
  return np.sum(np.absolute(x-y))

# pearson_correlation: recibe 2 arreglos numpy y
# retorna su correlación de pearson, las librerías adicionales
# sirven para evitar el mensaje de warning por si se diera el caso
# de que se intenta calcular la correlación y uno de los
# vectores 'x' o 'y' es constante, lo que resultaría en una división por cero
from scipy.stats.stats import pearsonr
import warnings
warnings.filterwarnings('error')
def pearson_correlation(x, y):
  try:
    return pearsonr(x, y)[0]
  # retorna 0 si se divide por 0 en el cálculo de la correlación
  except:
    return 0

# cosine_similarity: recibe 2 arreglos numpy y
# retorna su similitud de cosenos
def cosine_similarity(x, y):
  return np.dot(x, y)/(np.sqrt(np.dot(x,x))*np.sqrt(np.dot(y,y)))


In [87]:
# predict_rating: Función auxiliar para la función get_knn
# Recibe un diccionario de los vecinos más cercanos donde
# la clave es el id del usuario y el valor es la una distancia de similitud
# respectiva, también recibe el id de una película.
# Entonces, la función calcula el grado de influencia y retorna la predicción
# del rating para la película con dicho id
def predict_rating(knn, movieId):
  # total acumula la suma de las medidas de similitud de 
  # los vecinos más cercanos
  total = 0
  prediction = 0
  # Recorriendo el diccionario de vecinos más cercanos con sus
  # respectivas distancias
  for usrId, distance in knn.items():
    # solo se considera en el cálculo del grado de influencia a
    # aquellos usuarios que hayan calificado la película
    # a la que se intenta predecir el rating que recibiría
    if movieId in userRatings[usrId]:
      # se acumula en prediction el rating * medida de similitud
      prediction += userRatings[usrId][movieId]*distance
      total += distance
  # verificando que total no sea 0, que se da en el caso de 
  # que todas las medidas de similitud de los vecinos más cercanos sean 0
  if(total==0):
    return 0
  # se retorna la predicción
  return np.around(prediction/total, 1)
  
# get_knn: Función principal del programa, que tiene como parámetros
# user: id del usuario al que se le recomendarán películas
# k: número de vecinos más cercanos a considerar, si no se especifica,
# se utilizan 3 por defecto
# method: string con el nombre del método para el cálculo de
# la medida de similitud(pearson, euclidean, manhattan, cosine_sim)
# threshold: umbral para recomendar películas
def get_knn(user, k=10, method='pearson',threshold=3.5):
  # distances: diccionario con id de usuario como clave y
  # como valor tendrá la medida de similitud
  distances = {}
  dist = 0
  # Fuerza bruta para encontrar los vecinos más cercanos,
  # por lo que se requerirá calcular la similitud
  # con todos los usuarios
  for usrId in userRatings.keys():
    # no se calcula la similitud con uno mismo
    if (usrId == user):
      continue
    # obteniendo los arreglos con rating en común
    x, y = common_rating(user, usrId)
    # Si no tienen ningún rating en común
    # se devuelve 999999 para las distancias euclidianas y manhattan
    # y -2 para las correlación de pearson y la similitud de cosenos
    # Se asignan eso valores para que no se consideren a la
    # hora de recomendar películas
    if (x.size == 0):
      if (method=='euclidean' or method=='manhattan'):
        distances[usrId] = 999999
      else:
        distances[usrId] = -2
      continue
    # dependiendo del método escogido, se calcula la similitud
    if  (method=='euclidean'):
      dist = euclidean_distance(x, y)
    elif (method=='manhattan'):
      dist = manhattan_distance(x, y)
    elif (method=='pearson'):
      dist = pearson_correlation(x, y)
    elif (method=='cosine_sim'):
      dist = cosine_similarity(x, y)
    else:
      print('Método no válido')
      return
    # se almacena la distancia en el diccionario distances
    distances[usrId] = dist
  
  # Si es una medida de desimilitud se ordena de menor a mayor,
  # caso contrario se ordena de mayor a menor
  if(method=='euclidean' or method=='manhattan'):
    distances = {k: v for k, v in sorted(distances.items(), key=lambda item: item[1])}
  else:
    distances = {k: v for k, v in sorted(distances.items(), key=lambda item: item[1], reverse=True)}
      
  # solo se almacena en knn los k vecinos más cercanos
  knn = {}
  for usrId, distance in distances.items():
    if(k==0):
      break
    #print(usrId, distance)
    knn[usrId] = distance
    k -= 1
  
  # moviesFreq: diccionario con clave al id de la película
  # y con valor la frecuencia de la película que se repite entre los usuarios
  moviesFreq = {}
  for usrId in knn.keys():
    for movId, rating in userRatings[usrId].items():
      if (rating >= threshold):
        if movId in moviesFreq:
          moviesFreq[movId] += 1
        else:
          moviesFreq[movId] = 1

  # Se ordena moviesFreq de mayor a menor según la frecuencia
  # de las películas
  #print('Películas con mayor frecuencia entre los vecinos más cercanos:')
  moviesFreq = {k: v for k, v in sorted(moviesFreq.items(), key=lambda item: item[1], reverse=True)}

  # moviesName: diccionario con clave el id de una película y con valor una
  # tupla que representa la frecuencia de la película y el grado de influencia
  moviesName = {}
  for movId, value in moviesFreq.items():
    if movId not in userRatings[user]:
      moviesName[movieIdName[movId]] = (value, predict_rating(knn, movId))

  # imprimiendo las películas recomendadas a user
  #for key, value in moviesName.items():
    #print(key, value)


In [88]:
from random import randrange
# add_user: función que sirve para añadir un nuevo usuario al dataset
# que recibe como parámetro el número de películas que quiere calificar
def add_user(moviesCount=20):
  # su id de usuario será la próxima disponible
  newUsrId = len(userRatings)+1
  # ratings: diccionario con id de película como clave y el rating
  # a esta como valor
  ratings = {}
  for i in range(moviesCount):
    # obtenemos una fila aleatoria del dataset de películas
    # para que el usuario pueda calificar
    row = movieDf.loc[randrange(movieDf.shape[0])]

    # Solicitamos al usuario que califique(manualmente)
    #print("Ingrese una calificación para la película ", end="")
    #print(row['title'],end=":\n")
    #ratings[row['movieId']] = float(input())

    # Dar un rating aleatorio entre 1 y 5(automático)
    ratings[row['movieId']] = randrange(1,5)
  # Finalmente agregamos al usuario al dataset
  userRatings[newUsrId] = ratings
  # retornamos su id
  return newUsrId

usrId = add_user(moviesCount=100)
# Le recomendamos películas al nuevo usuario
get_knn(usrId, k=7, method='manhattan')

# Midiendo tiempos de las funciones

In [89]:
%timeit x,y = common_rating(1, 2)

10000 loops, best of 5: 26.6 µs per loop


In [91]:
x,y = common_rating(1, 2)

In [92]:
%timeit euclidean_distance(x, y)

The slowest run took 19.58 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 9.9 µs per loop


In [93]:
%timeit manhattan_distance(x, y)

The slowest run took 15.03 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 6.28 µs per loop


In [94]:
%timeit pearson_correlation(x, y)

The slowest run took 7.65 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 72.7 µs per loop


In [95]:
%timeit cosine_similarity(x, y)

The slowest run took 9.89 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 10.7 µs per loop


In [96]:
%timeit get_knn(580, k=7, method='manhattan')

10 loops, best of 5: 148 ms per loop


# Determinando K óptimo

In [15]:
get_knn(usrId, k=4, method='manhattan')

2819 3
1621 5
5159 5
2133 6
Películas con mayor frecuencia entre los vecinos más cercanos:
U-571 (2000) (3, 4.2)
Star Wars: Episode IV - A New Hope (1977) (2, 5.0)
Rules of Engagement (2000) (2, 3.8)
Frequency (2000) (2, 3.7)
Erin Brockovich (2000) (1, 4.0)
Cider House Rules, The (1999) (1, 5.0)
What Planet Are You From? (2000) (1, 4.0)
Supernova (2000) (1, 2.8)
Mission to Mars (2000) (1, 3.1)
I Dreamed of Africa (2000) (1, 5.0)
Platoon (1986) (1, 5.0)
Welcome to the Dollhouse (1995) (1, 4.0)
Final Destination (2000) (1, 3.5)
Lost World: Jurassic Park, The (1997) (1, 2.9)
Gladiator (2000) (1, 3.5)
Wizard of Oz, The (1939) (1, 5.0)
Payback (1999) (1, 4.0)
Silence of the Lambs, The (1991) (1, 5.0)
Peacemaker, The (1997) (1, 5.0)
Any Given Sunday (1999) (1, 4.0)
Men in Black (1997) (1, 4.0)
Committed (2000) (1, 5.0)
Rudy (1993) (1, 4.0)


In [16]:
get_knn(usrId, k=5, method='manhattan')

2819 3
1621 5
5159 5
2133 6
2357 6
Películas con mayor frecuencia entre los vecinos más cercanos:
U-571 (2000) (4, 4.4)
Rules of Engagement (2000) (3, 3.9)
Erin Brockovich (2000) (2, 4.0)
Star Wars: Episode IV - A New Hope (1977) (2, 5.0)
Mission to Mars (2000) (2, 3.5)
I Dreamed of Africa (2000) (2, 4.3)
Final Destination (2000) (2, 4.1)
Frequency (2000) (2, 3.5)
Silence of the Lambs, The (1991) (2, 5.0)
Any Given Sunday (1999) (2, 4.0)
Cider House Rules, The (1999) (1, 3.7)
What Planet Are You From? (2000) (1, 4.0)
Supernova (2000) (1, 2.8)
Platoon (1986) (1, 5.0)
Welcome to the Dollhouse (1995) (1, 4.0)
Lost World: Jurassic Park, The (1997) (1, 2.9)
Gladiator (2000) (1, 3.5)
Wizard of Oz, The (1939) (1, 5.0)
Payback (1999) (1, 4.0)
Peacemaker, The (1997) (1, 5.0)
Men in Black (1997) (1, 4.0)
Committed (2000) (1, 5.0)
Rudy (1993) (1, 4.0)
Scream 3 (2000) (1, 5.0)
Snow Day (2000) (1, 4.0)
Skulls, The (2000) (1, 3.5)
Hollow Man (2000) (1, 3.5)
Magnolia (1999) (1, 4.0)
Reindeer Games (2

In [17]:
get_knn(usrId, k=6, method='manhattan')

2819 3
1621 5
5159 5
2133 6
2357 6
1927 8
Películas con mayor frecuencia entre los vecinos más cercanos:
U-571 (2000) (4, 4.4)
Rules of Engagement (2000) (3, 3.9)
Erin Brockovich (2000) (2, 4.0)
Star Wars: Episode IV - A New Hope (1977) (2, 5.0)
Mission to Mars (2000) (2, 3.5)
I Dreamed of Africa (2000) (2, 4.3)
Final Destination (2000) (2, 4.1)
Frequency (2000) (2, 3.5)
Silence of the Lambs, The (1991) (2, 5.0)
Any Given Sunday (1999) (2, 4.0)
Men in Black (1997) (2, 4.0)
Cider House Rules, The (1999) (1, 3.7)
What Planet Are You From? (2000) (1, 4.0)
Supernova (2000) (1, 2.8)
Platoon (1986) (1, 5.0)
Welcome to the Dollhouse (1995) (1, 4.0)
Lost World: Jurassic Park, The (1997) (1, 2.9)
Gladiator (2000) (1, 3.5)
Wizard of Oz, The (1939) (1, 5.0)
Payback (1999) (1, 4.0)
Peacemaker, The (1997) (1, 5.0)
Committed (2000) (1, 5.0)
Rudy (1993) (1, 4.0)
Scream 3 (2000) (1, 5.0)
Snow Day (2000) (1, 4.0)
Skulls, The (2000) (1, 3.5)
Hollow Man (2000) (1, 3.5)
Magnolia (1999) (1, 4.0)
Reindeer G

In [22]:
get_knn(usrId, k=7, method='manhattan')

2819 3
1621 5
5159 5
2133 6
2357 6
1927 8
3133 8
Películas con mayor frecuencia entre los vecinos más cercanos:
U-571 (2000) (4, 4.4)
Erin Brockovich (2000) (3, 4.0)
Rules of Engagement (2000) (3, 3.9)
Star Wars: Episode IV - A New Hope (1977) (2, 5.0)
Mission to Mars (2000) (2, 3.5)
I Dreamed of Africa (2000) (2, 4.3)
Final Destination (2000) (2, 4.1)
Frequency (2000) (2, 3.5)
Silence of the Lambs, The (1991) (2, 5.0)
Any Given Sunday (1999) (2, 4.0)
Men in Black (1997) (2, 4.0)
Patriot, The (2000) (2, 4.0)
Full Monty, The (1997) (2, 4.0)
Cider House Rules, The (1999) (1, 3.7)
What Planet Are You From? (2000) (1, 4.0)
Supernova (2000) (1, 2.8)
Platoon (1986) (1, 5.0)
Welcome to the Dollhouse (1995) (1, 4.0)
Lost World: Jurassic Park, The (1997) (1, 2.9)
Gladiator (2000) (1, 3.3)
Wizard of Oz, The (1939) (1, 5.0)
Payback (1999) (1, 4.0)
Peacemaker, The (1997) (1, 5.0)
Committed (2000) (1, 5.0)
Rudy (1993) (1, 4.0)
Scream 3 (2000) (1, 3.9)
Snow Day (2000) (1, 4.0)
Skulls, The (2000) (1,

In [19]:
get_knn(usrId, k=10, method='manhattan')

2819 3
1621 5
5159 5
2133 6
2357 6
1927 8
3133 8
3542 8
907 9
1363 9
Películas con mayor frecuencia entre los vecinos más cercanos:
U-571 (2000) (4, 4.4)
Erin Brockovich (2000) (3, 3.7)
Star Wars: Episode IV - A New Hope (1977) (3, 4.5)
Rules of Engagement (2000) (3, 3.9)
Silence of the Lambs, The (1991) (3, 4.6)
Mission to Mars (2000) (2, 3.5)
I Dreamed of Africa (2000) (2, 4.3)
Final Destination (2000) (2, 4.1)
Gladiator (2000) (2, 3.5)
Frequency (2000) (2, 3.5)
Any Given Sunday (1999) (2, 4.0)
Men in Black (1997) (2, 4.0)
Skulls, The (2000) (2, 3.7)
Magnolia (1999) (2, 4.6)
Reindeer Games (2000) (2, 3.7)
Patriot, The (2000) (2, 4.0)
Full Monty, The (1997) (2, 4.0)
Cider House Rules, The (1999) (1, 3.7)
What Planet Are You From? (2000) (1, 4.0)
Supernova (2000) (1, 2.8)
Platoon (1986) (1, 5.0)
Welcome to the Dollhouse (1995) (1, 4.0)
Lost World: Jurassic Park, The (1997) (1, 2.9)
Wizard of Oz, The (1939) (1, 5.0)
Payback (1999) (1, 4.0)
Peacemaker, The (1997) (1, 5.0)
Committed (2000

#Buscando medida de similitud óptima

In [26]:
get_knn(usrId, k=7, method='euclidean')

2819 1.7320508075688772
1621 2.23606797749979
2357 3.1622776601683795
2133 3.4641016151377544
3133 3.4641016151377544
1091 3.605551275463989
5159 3.605551275463989
Películas con mayor frecuencia entre los vecinos más cercanos:
U-571 (2000) (4, 4.4)
Erin Brockovich (2000) (3, 4.0)
Rules of Engagement (2000) (3, 3.8)
Star Wars: Episode IV - A New Hope (1977) (2, 5.0)
Mission to Mars (2000) (2, 3.4)
I Dreamed of Africa (2000) (2, 4.4)
Final Destination (2000) (2, 4.0)
Silence of the Lambs, The (1991) (2, 5.0)
Any Given Sunday (1999) (2, 4.0)
Patriot, The (2000) (2, 4.0)
Frequency (2000) (2, 3.6)
Cider House Rules, The (1999) (1, 3.7)
What Planet Are You From? (2000) (1, 4.0)
Supernova (2000) (1, 2.6)
Platoon (1986) (1, 5.0)
Welcome to the Dollhouse (1995) (1, 4.0)
Lost World: Jurassic Park, The (1997) (1, 2.9)
Gladiator (2000) (1, 3.2)
Scream 3 (2000) (1, 4.0)
Snow Day (2000) (1, 4.0)
Skulls, The (2000) (1, 3.5)
Hollow Man (2000) (1, 3.6)
Magnolia (1999) (1, 4.0)
Reindeer Games (2000) (1,

In [27]:
get_knn(usrId, k=7, method='manhattan')

2819 3
1621 5
5159 5
2133 6
2357 6
1927 8
3133 8
Películas con mayor frecuencia entre los vecinos más cercanos:
U-571 (2000) (4, 4.4)
Erin Brockovich (2000) (3, 4.0)
Rules of Engagement (2000) (3, 3.9)
Star Wars: Episode IV - A New Hope (1977) (2, 5.0)
Mission to Mars (2000) (2, 3.5)
I Dreamed of Africa (2000) (2, 4.3)
Final Destination (2000) (2, 4.1)
Frequency (2000) (2, 3.5)
Silence of the Lambs, The (1991) (2, 5.0)
Any Given Sunday (1999) (2, 4.0)
Men in Black (1997) (2, 4.0)
Patriot, The (2000) (2, 4.0)
Full Monty, The (1997) (2, 4.0)
Cider House Rules, The (1999) (1, 3.7)
What Planet Are You From? (2000) (1, 4.0)
Supernova (2000) (1, 2.8)
Platoon (1986) (1, 5.0)
Welcome to the Dollhouse (1995) (1, 4.0)
Lost World: Jurassic Park, The (1997) (1, 2.9)
Gladiator (2000) (1, 3.3)
Wizard of Oz, The (1939) (1, 5.0)
Payback (1999) (1, 4.0)
Peacemaker, The (1997) (1, 5.0)
Committed (2000) (1, 5.0)
Rudy (1993) (1, 4.0)
Scream 3 (2000) (1, 3.9)
Snow Day (2000) (1, 4.0)
Skulls, The (2000) (1,

In [28]:
get_knn(usrId, k=7, method='pearson')

4515 0.7905694150420948
317 0.7218397547786818
5069 0.705642285072797
2815 0.6981033097668207
5315 0.6665826983223992
3286 0.6543811821755866
1456 0.6484603870837242
Películas con mayor frecuencia entre los vecinos más cercanos:
Wizard of Oz, The (1939) (2, 4.0)
Gone with the Wind (1939) (2, 5.0)
Star Wars: Episode IV - A New Hope (1977) (2, 4.0)
Star Wars: Episode V - The Empire Strikes Back (1980) (2, 4.0)
Graduate, The (1967) (2, 4.5)
Usual Suspects, The (1995) (2, 4.5)
African Queen, The (1951) (1, 5.0)
Goldfinger (1964) (1, 4.0)
Treasure of the Sierra Madre, The (1948) (1, 5.0)
Great Escape, The (1963) (1, 5.0)
Butch Cassidy and the Sundance Kid (1969) (1, 5.0)
Singin' in the Rain (1952) (1, 5.0)
Mrs. Doubtfire (1993) (1, 5.0)
M*A*S*H (1970) (1, 4.0)
Close Encounters of the Third Kind (1977) (1, 5.0)
Chariots of Fire (1981) (1, 4.0)
Driving Miss Daisy (1989) (1, 4.0)
Laura (1944) (1, 5.0)
Cook the Thief His Wife & Her Lover, The (1989) (1, 5.0)
Grifters, The (1990) (1, 5.0)
Medite

In [29]:
get_knn(usrId, k=7, method='cosine_sim')

3783 0.9838699100999074
1363 0.9782806431359609
3238 0.9779008859218913
1621 0.9756379699771127
2819 0.9740492440449616
1351 0.9739826654742461
2357 0.9739318301999084
Películas con mayor frecuencia entre los vecinos más cercanos:
Rules of Engagement (2000) (3, 4.3)
Babe (1995) (2, 4.5)
Magnolia (1999) (2, 4.5)
Christmas Story, A (1983) (2, 5.0)
Star Wars: Episode V - The Empire Strikes Back (1980) (2, 4.0)
Final Destination (2000) (2, 4.5)
Erin Brockovich (2000) (2, 4.0)
Mission to Mars (2000) (2, 4.5)
U-571 (2000) (2, 4.3)
I Dreamed of Africa (2000) (2, 4.5)
Cool Hand Luke (1967) (1, 4.0)
Little Big Man (1970) (1, 4.0)
Bringing Up Baby (1938) (1, 5.0)
Butch Cassidy and the Sundance Kid (1969) (1, 4.0)
Prince of Egypt, The (1998) (1, 4.0)
Network (1976) (1, 4.0)
Tommy Boy (1995) (1, 4.0)
Roger & Me (1989) (1, 5.0)
Groundhog Day (1993) (1, 5.0)
Commitments, The (1991) (1, 5.0)
Thin Red Line, The (1998) (1, 5.0)
Sheltering Sky, The (1990) (1, 4.0)
Player, The (1992) (1, 5.0)
Hoop Dreams